In [81]:
import midi_functions
import numpy as np

from keras.models import Sequential
from keras.layers import LSTM, Dense, Activation, Dropout, Flatten
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

import pydot as pyd
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

In [69]:
song = './Bach-1.mid'
features = midi_functions.get_features(path=song)

input_length = 10 #How many past notes we want to give the LSTM to predict the next one
X, Y = midi_functions.get_XY(features,n_notes=input_length)
X = X[:,np.arange(0,4 * input_length,4)]#gets only the notes and discards velocity/time info
Y = Y[:,0]

#How many notes are present?
sorted_notes = np.copy(Y)
sorted_notes.sort()
sorted_notes = np.unique(sorted_notes)

number_of_different_notes = len(sorted_notes)
print('Number of different notes: {}'.format(number_of_different_notes))

#Convert Y into the one-hot-representation because I think that is the only thing that works with softmax representation
for i,note in enumerate(Y): #make all the notes a number between 0 and number_of_different_notes
    Y[i] = np.where(sorted_notes == note)[0]
Y = np_utils.to_categorical(Y)
print(Y.shape)


Number of different notes: 38
(699, 38)


In [37]:
#randomize the two arrays and split into training and validation
n = len(X)
permutation = np.random.permutation(n)
X = X[permutation]
Y = Y[permutation]

X_val = X[:200]
Y_val = Y[:200]
X_train = X[200:]
Y_train = Y[200:]

In [86]:

#input_dimensions = (input_length,1)

n_hidden_units = number_of_different_notes #this should probably be the number of different notes since it is also the output of softmax

model = Sequential()
model.add(LSTM(
    n_hidden_units,
    input_shape=((input_length,)),
    return_sequences=False
))
model.add(Dropout(0.3))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
# Load the weights to each node
#model.load_weights('weights.hdf5')


#plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)
print(model.summary())
#print(model.get_weights())

ValueError: Input 0 is incompatible with layer lstm_14: expected ndim=3, found ndim=2

In [84]:
filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"    
checkpoint = ModelCheckpoint(
    filepath, monitor='loss', 
    verbose=0,        
    save_best_only=True,        
    mode='min'
)    
callbacks_list = [checkpoint]     
model.fit(X_train, Y_train, epochs=100, batch_size=64, callbacks=callbacks_list)

ValueError: Error when checking input: expected lstm_12_input to have 3 dimensions, but got array with shape (499, 10)